# Set-Up

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('../data/data.db')
c = conn.cursor()

def execute_statement(statement):
    c.execute(statement)
    res = c.fetchall()
    column_names = [description[0] for description in c.description]
    return pd.DataFrame(res, columns=column_names)

# Countries of Europe
In this first section we will create a table in the database containing all of the country codes for territories that are partially or completely in Europe. Each of these country codes also also shown in the table below.

| Code | Country |
|---|---|
| AD | Andorra |
| AL | Albania |
| AM | Armenia |
| AT | Austria |
| AX | Aland Islands |
| AZ | Azerbaijan |
| BA | Bosnia and Herzegovina |
| BE | Belgium |
| BG | Bulgaria |
| BY | Belarus |
| CH | Switzerland |
| CS | Former Serbia and Montenegro |
| CY | Cyprus |
| CZ | Czech Republic |
| DD | Former East Germany |
| DE | Germany |
| DK | Denmark |
| EE | Estonia |
| ES | Spain |
| FI | Finland |
| FO | Faroe Islands |
| FR | France |
| GB | United Kingdom of Great Britain and Northern Ireland |
| GE | Georgia |
| GG | Guernsey |
| GI | Gibraltar |
| GR | Greece |
| HR | Croatia |
| HU | Hungary |
| IE | Ireland |
| IM | Isle of Man |
| IS | Iceland |
| IT | Italy |
| JE | Jersey |
| LI | Liechtenstein |
| LT | Lithuania |
| LU | Luxembourg |
| LV | Latvia |
| MC | Monaco |
| MD | Republic of Moldova |
| ME | Montenegro |
| MT | Malta |
| NL | Netherlands |
| NO | Norway |
| PL | Poland |
| PT | Portugal |
| RO | Romania |
| RS | Serbia |
| RU | Russian Federation |
| SE | Sweden |
| SI | Slovenia |
| SK | Slovakia |
| SM | San Marino |
| SU | Former Soviet Union |
| TR | Turkey |
| UA | Ukraine |
| VA | Holy See |
| XE | Europe |
| YU | Former Yugoslavia |
| ZC | Former Czechoslovakia |

In [3]:
# Create the table
c.execute('''CREATE TABLE IF NOT EXISTS EuropeTerritories
             (
                 Code varchar(3),
                 Territory varchar(255)
             )''')

# Get a Python list of territories
countries = [('AD', 'Andorra'), ('AL', 'Albania'), ('AM', 'Armenia'), ('AT', 'Austria'), ('AX', 'Aland Islands'),
             ('AZ', 'Azerbaijan'), ('BA', 'Bosnia and Herzegovina'), ('BE', 'Belgium'), ('BG', 'Bulgaria'),
             ('BY', 'Belarus'), ('CH', 'Switzerland'), ('CS', 'Former Serbia and Montenegro'), ('CY', 'Cyprus'),
             ('CZ', 'Czech Republic'), ('DD', 'Former East Germany'), ('DE', 'Germany'), ('DK', 'Denmark'),
             ('EE', 'Estonia'), ('ES', 'Spain'), ('FI', 'Finland'), ('FO', 'Faroe Islands'), ('FR', 'France'),
             ('GB', 'United Kingdom of Great Britain and Northern Ireland'), ('GE', 'Georgia'), ('GG', 'Guernsey'),
             ('GI', 'Gibraltar'), ('GR', 'Greece'), ('HR', 'Croatia'), ('HU', 'Hungary'), ('IE', 'Ireland'),
             ('IM', 'Isle of Man'), ('IS', 'Iceland'), ('IT', 'Italy'), ('JE', 'Jersey'), ('LI', 'Liechtenstein'),
             ('LT', 'Lithuania'), ('LU', 'Luxembourg'), ('LV', 'Latvia'), ('MC', 'Monaco'), ('MD', 'Republic of Moldova'),
             ('ME', 'Montenegro'), ('MT', 'Malta'), ('NL', 'Netherlands'), ('NO', 'Norway'), ('PL', 'Poland'),
             ('PT', 'Portugal'), ('RO', 'Romania'), ('RS', 'Serbia'), ('RU', 'Russian Federation'), ('SE', 'Sweden'),
             ('SI', 'Slovenia'), ('SK', 'Slovakia'), ('SM', 'San Marino'), ('SU', 'Former Soviet Union'),
             ('TR', 'Turkey'), ('UA', 'Ukraine'), ('VA', 'Holy See'), ('XE', 'Europe'), ('YU', 'Former Yugoslavia'),
             ('ZC', 'Former Czechoslovakia')]

# Add countries to the table
for pair in countries:
    c.execute('INSERT INTO EuropeTerritories VALUES("' + pair[0] + '", "' + pair[1] + '")')

This extra step is to check that we have managed to insert all 60 codes into our new EuropeTerritories table.

In [4]:
execute_statement('SELECT COUNT(*) FROM EuropeTerritories')

COUNT(*)
0        60

# Simple Exploration
To begin, we will do some simple SQL exploration.

The cell below will find the importing countries of Europe that are most represented in the CITES table.

In [5]:
execute_statement('''SELECT RANK() OVER (ORDER BY COUNT(*) DESC) AS "Rank",
                            Territory,
                            COUNT(*) AS "Import Count"
                     FROM EuropeTerritories
                     INNER JOIN cites
                     ON EuropeTerritories.Code = cites.Importer
                     GROUP BY cites.Importer
                     ORDER BY COUNT(*) DESC
                     LIMIT 10''')

Rank                                          Territory  Import Count
0     1                                              Spain         14870
1     2  United Kingdom of Great Britain and Northern I...         14427
2     3                                            Germany         12106
3     4                                        Netherlands         11069
4     5                                           Portugal          9237
5     6                                              Italy          9206
6     7                                             France          8608
7     8                                        Switzerland          7551
8     9                                             Norway          7041
9    10                                            Belgium          4624

As we can see from the results, Spain is the country with most import records in the CITES table - closely followed by the United Kingdom.

We will now repeat the same with exports.

In [6]:
execute_statement('''SELECT RANK() OVER (ORDER BY COUNT(*) DESC) AS "Rank",
                            Territory,
                            COUNT(*) AS "Export Count"
                     FROM EuropeTerritories
                     INNER JOIN cites
                     ON EuropeTerritories.Code = cites.Exporter
                     GROUP BY cites.Exporter
                     ORDER BY COUNT(*) DESC
                     LIMIT 10''')

Rank                                          Territory  Export Count
0     1  United Kingdom of Great Britain and Northern I...         56596
1     2                                        Netherlands         35646
2     3                                              Spain         35288
3     4                                            Germany         33304
4     5                                            Belgium         32654
5     6                                     Czech Republic         18337
6     7                                            Austria         15439
7     8                                            Denmark          7866
8     9                                             France          7051
9    10                                        Switzerland          6529

The United Kingdom is the country that is most represented in our table for exports, with over 56,000 export records. The Netherlands is in 2nd and Spain is in 3rd, both with just over 35,000 export records. Many of the countries in our top 10 exporters were also in the top 10 importers list, we would intuitively expect some correlation between the rankings of the two.

We can also look at what species are being imported and exported. Of interest might be live species. In the cell below we will find the live species that are most exported (in terms of quantity, not number of records) from Europe since 2015.

In [7]:
execute_statement('''SELECT RANK() OVER (ORDER BY SUM(Quantity) DESC) AS "Rank",
                            Taxon,
                            SUM(Quantity) AS "Export Count",
                            Family
                     FROM cites
                     WHERE Exporter IN (SELECT Code FROM EuropeTerritories) AND Term = "live" AND Year >= 2015
                     GROUP BY Taxon
                     ORDER BY SUM(Quantity) DESC
                     LIMIT 5''')

Rank                   Taxon  Export Count       Family
0     1  Psephotus haematonotus       77178.0  Psittacidae
1     2     Platycercus elegans       72023.0  Psittacidae
2     3      Agapornis fischeri       59145.0  Psittacidae
3     4     Platycercus eximius       57991.0  Psittacidae
4     5    Agapornis personatus       46141.0  Psittacidae

The most exported species is *Psephotus haematonotus*, or the red-rumped parrot. Although easy to breed, this species is actually native to Australia, and so further exploration of this result might yield interesting results. In second place is the crimson rosella (*Platycercus elegans*), and in third place is Fischer's lovebird (*Agapornis fischeri*). The top five export species is actually completely dominated by parrots.

Now, we will try the same query but for imports.

In [8]:
execute_statement('''SELECT RANK() OVER (ORDER BY SUM(Quantity) DESC) AS "Rank",
                            Taxon,
                            SUM(Quantity) AS "Import Count",
                            Family
                     FROM cites
                     WHERE Importer IN (SELECT Code FROM EuropeTerritories) AND Term = "live" AND Year >= 2015
                     GROUP BY Taxon
                     ORDER BY SUM(Quantity) DESC
                     LIMIT 5''')

Rank                  Taxon  Import Count       Family
0     1     Agapornis fischeri       60612.0  Psittacidae
1     2   Agapornis personatus       27429.0  Psittacidae
2     3    Psittacus erithacus       24991.0  Psittacidae
3     4    Platycercus eximius       15173.0  Psittacidae
4     5  Poicephalus senegalus        9828.0  Psittacidae

Again, we find that the top 5 list is completely dominated by parrots. We will repeat both queries with the Psittacidae family excluded, to see the most popular non-parrot species too.

In [9]:
execute_statement('''SELECT RANK() OVER (ORDER BY SUM(Quantity) DESC) AS "Rank",
                            Taxon,
                            SUM(Quantity) AS "Export Count",
                            Family
                     FROM cites
                     WHERE Exporter IN (SELECT Code FROM EuropeTerritories)
                         AND Term = "live"
                         AND Year >= 2015
                         AND Family <> "Psittacidae"
                     GROUP BY Taxon
                     ORDER BY SUM(Quantity) DESC
                     LIMIT 5''')

Rank               Taxon  Export Count       Family
0     1        Falco hybrid       28502.0   Falconidae
1     2       Columba livia       18748.0   Columbidae
2     3    Falco rusticolus       14143.0   Falconidae
3     4  Lonchura oryzivora        8931.0  Estrildidae
4     5    Falco peregrinus        8499.0   Falconidae

In [10]:
execute_statement('''SELECT RANK() OVER (ORDER BY SUM(Quantity) DESC) AS "Rank",
                            Taxon,
                            SUM(Quantity) AS "Import Count",
                            Family
                     FROM cites
                     WHERE Importer IN (SELECT Code FROM EuropeTerritories)
                         AND Term = "live"
                         AND Year >= 2015
                         AND Family <> "Psittacidae"
                     GROUP BY Taxon
                     ORDER BY SUM(Quantity) DESC
                     LIMIT 5''')

Rank               Taxon  Import Count       Family
0     1    Falco rusticolus        3332.0   Falconidae
1     2       Columba livia        1196.0   Columbidae
2     3   Cacatua ducorpsii         890.0   Cacatuidae
3     4    Falco peregrinus         692.0   Falconidae
4     5  Lonchura oryzivora         678.0  Estrildidae

With parrots removed, we can see that falcons are also popular as imports and exports - the *Falco hybrid* is particularly popular as an export species, while the *Falco rusticolus* is particularly popular as an import species. The rock dove (*Columba livia*), or possibly the common domestic subspecies of *Columba livia domestica*, is also a popular import and export.

# Country By Country
A country-by-country analysis will soon appear.

# Clean-Up
To clean up the database and ensure that it does not interfere with other notebooks, we will delete the EuropeTerritories table.

In [11]:
c.execute('DROP TABLE EuropeTerritories')